In [2]:
import pandas as pd

import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.stem import LancasterStemmer 

nltk.download("stopwords")

import string
import re

import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

from sklearn.linear_model import SGDClassifier

import numpy as np

from scipy import sparse
import datetime

import s3fs

# import lch_proprietary.ml_modeling as ml 

from sklearn.feature_extraction.text import TfidfVectorizer

import io

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/natashamathur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
test = pd.read_csv("s3://advancedml-koch-mathur-hinkson/test.csv")

In [ ]:
train = pd.read_csv("s3://advancedml-koch-mathur-hinkson/train.csv")

In [3]:
train = pd.read_csv("train.csv")

Split train.csv into train (80%) and validation sets (20%).

In [4]:
validation_set = train[:360000]

In [5]:
train_set = train[360000:]

To speed up process, create small samples of the train_set on which to run NB.  Ensure that samples are iid by replacing after each draw.

In [6]:
train_sample1 = train_set.sample(frac=0.05, replace=True, random_state=1)

In [7]:
train_sample1.shape

(72244, 45)

In [8]:
ls = LancasterStemmer()
ps = PorterStemmer() 

sw = set(stopwords.words('english'))
sw.add('')

def clean_text(text, stemming=None, remove_sw = True):

    t = text.replace("-", " ").split(" ")
    
    t = [w.lower() for w in t]
    
    if remove_sw == True:
        t = [w for w in t if w not in sw]
    
    if stemming == None:
        pass;
    elif stemming == "Porter":
        t = [ps.stem(w) for w in t]
    elif stemming == "Lancaster":
        t = [ls.stem(w) for w in t]
    else:
        print("Please enter a valid stemming type")
        
    t = [w.strip(string.punctuation) for w in t]


    return ' '.join(t)

In [9]:
def add_text_cleaning_cols(df):
    print(datetime.datetime.now())
    
    df['split'] = df["comment_text"].apply(lambda x: x.split(" "))
    df['cleaned_w_stopwords'] = df["comment_text"].apply(clean_text,args=(None,False),)

    print(datetime.datetime.now())
    df['cleaned_no_stem'] = df["comment_text"].apply(clean_text,)
    df['cleaned_porter'] = df["comment_text"].apply(clean_text,args=("Porter",),)
    df['cleaned_lancaster'] = df["comment_text"].apply(clean_text,args=("Lancaster",),)

    print(datetime.datetime.now())

    df['perc_upper'] = df["comment_text"].apply(lambda x: round((len(re.findall(r'[A-Z]',x)) / len(x)), 3))

    df['num_exclam'] = df["comment_text"].apply(lambda x:(len(re.findall(r'!',x))))
    
    df['num_words'] = df["split"].apply(lambda x: len(x))
    print(datetime.datetime.now())

    df['perc_stopwords'] = round((df.num_words - df['cleaned_no_stem'].apply(lambda x: len(x)))/df.num_words,3) 
    
    df['num_upper_words'] = df["split"].apply(lambda x: sum(map(str.isupper, x)) 
)
    print(datetime.datetime.now())

    
    

In [10]:
add_text_cleaning_cols(train_sample1)

2019-05-26 07:24:10.342159
2019-05-26 07:24:12.441744
2019-05-26 07:25:21.998794
2019-05-26 07:25:22.532530
2019-05-26 07:25:22.856904


In [11]:
train_sample1.columns

Index(['id', 'target', 'comment_text', 'severe_toxicity', 'obscene',
       'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date',
       'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow',
       'sad', 'likes', 'disagree', 'sexual_explicit',
       'identity_annotator_count', 'toxicity_annotator_count', 'split',
       'cleaned_w_stopwords', 'cleaned_no_stem', 'cleaned_porter',
       'cleaned_lancaster', 'perc_upper', 'num_exclam', 'num_words',
       'perc_stopwords', 'num_upper_words'],
      dtype='object')

Pickle the dataset and send to s3 bucket:

In [ ]:

pickle_buffer = io.BytesIO()
s3_resource = boto3.resource('s3')
bucket='advancedml-koch-mathur-hinkson'
key='preprocessed_train_sample_50pct.pkl'

# test.to_pickle(pickle_buffer)
# s3_resource.Object(bucket, 'full_preprocessed_test.pkl').put(Body=pickle_buffer.getvalue())

test.to_pickle(key)
s3_resource.Object(bucket,key).put(Body=open(key, 'rb'))


In [12]:
addtl_cols = ['perc_upper', 'num_exclam', 'num_words', 'perc_stopwords', 'num_upper_words']
#addtl_cols = ['perc_upper']

In [13]:
train_sample1.shape

(72244, 55)

In [14]:
train_sample1.head(5)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,split,cleaned_w_stopwords,cleaned_no_stem,cleaned_porter,cleaned_lancaster,perc_upper,num_exclam,num_words,perc_stopwords,num_upper_words
488037,842255,0.0,"What, you don't think the earth is at least 8,...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,"[What,, you, don't, think, the, earth, is, at,...","what you don't think the earth is at least 8,0...","what think earth least 8,000 years old","what think earth least 8,000 year old","what think ear least 8,000 year old",0.017,0,12,-2.167,0
851755,5162644,0.0,"IMO one reason is the ""degree of foresight"". ...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,"[IMO, one, reason, is, the, ""degree, of, fores...",imo one reason is the degree of foresight tho...,imo one reason degree foresight less foresight...,imo one reason degre foresight less foresight ...,imo on reason degree foresight less foresight ...,0.019,0,53,-1.660,1
830924,5137657,0.0,""" Oh- and throw in some more ""racist"" and ""mys...",0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,"["", Oh-, and, throw, in, some, more, ""racist"",...",oh and throw in some more racist and mysogen...,oh throw racist mysogeny name calls supporter...,oh throw racist mysogeny name call support wo...,oh throw racist mysogeny nam cal support wond...,0.013,0,36,-3.306,0
851263,5162070,0.0,If anyone understands the subtle intricacies o...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,"[If, anyone, understands, the, subtle, intrica...",if anyone understands the subtle intricacies o...,anyone understands subtle intricacies saying o...,anyon understand subtl intricaci say one thing...,anyon understand subtl int say on thing vot an...,0.015,0,34,-3.441,0
1196489,5578177,0.0,"""klu klux klan""?",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"[""klu, klux, klan""?]",klu klux klan,klu klux klan,klu klux klan,klu klux klan,0.000,0,3,-3.333,0


Due to memory issues, we needed to use a smaller training set.  We used a random iid sample of half of the train_sample1 frame to train NB model:

In [16]:
subset_train_sample1 = train_sample1.sample(frac=0.5, replace=True, random_state=1)

In [17]:
def assign_toxicity(target):
    if target > 0.5:
        return 1
    else:
        return 0

Calculated a binary "toxic" or "non toxic" class designation based on the target score

In [18]:
subset_train_sample1['cat'] = subset_train_sample1.target.apply(assign_toxicity,)

In [19]:
toxic = subset_train_sample1[subset_train_sample1.cat == 1]
nontoxic = subset_train_sample1[subset_train_sample1.cat == 0]

In [20]:
subset_train_sample1.shape, toxic.shape, nontoxic.shape

((36122, 56), (2204, 56), (33918, 56))

Reshaping the dataset to be include an equal number of toxic and nontoxic samples

In [21]:
sample_df = toxic.append(toxic).append(nontoxic.sample(2204*2))
sample_df = sample_df.sample(frac=1).reset_index(drop=True)

print(sample_df.cat.value_counts())


1    4408
0    4408
Name: cat, dtype: int64


In [33]:
sample_df[sample_df.identity_attack > .5]

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,cleaned_w_stopwords,cleaned_no_stem,cleaned_porter,cleaned_lancaster,perc_upper,num_exclam,num_words,perc_stopwords,num_upper_words,cat
5,5141102,0.800000,"So you have a problem with white people, you s...",0.000000,0.000000,0.800000,0.200000,0.000000,0.00,0.000000,...,so you have a problem with white people you se...,problem white people seem think trump supporte...,problem white people seem think trump support ...,problem whit people seem think trump support r...,0.026,0,97,-2.588,2,1
9,786414,0.642857,So obomba is the black monkey. Got it,0.100000,0.028571,0.585714,0.614286,0.000000,0.00,0.166667,...,so obomba is the black monkey got it,obomba black monkey got,obomba black monkey got,obomb black monkey got,0.054,0,8,-1.875,0,1
10,5490705,0.837500,Karem is a fag,0.125000,0.550000,0.562500,0.812500,0.000000,0.00,0.000000,...,karem is a fag,karem fag,karem fag,karem fag,0.071,0,4,-1.250,0,1
19,707353,0.700000,"No, he means right-wing religious-extremist il...",0.000000,0.100000,0.700000,0.500000,0.000000,NaN,NaN,...,no he means right wing religious extremist ill...,no means right wing religious extremist illega...,no mean right wing religi extremist illeg isra...,no mean right wing religy extrem illeg israel ...,0.021,0,21,-4.238,0,1
61,6192611,0.878378,One day I hope science can understand the path...,0.108108,0.243243,0.648649,0.810811,0.000000,0.00,0.000000,...,one day i hope science can understand the path...,one day hope science understand pathology blac...,one day hope scienc understand patholog black ...,on day hop sci understand patholog black vot s...,0.023,0,20,-4.350,1,1
63,749829,0.893939,gay probably poor also otherwise he could of m...,0.151515,0.803030,0.590909,0.848485,0.015152,0.00,0.000000,...,gay probably poor also otherwise he could of m...,gay probably poor also otherwise could moved l...,gay probabl poor also otherwis could move like...,gay prob poor also otherw could mov lik bitch ...,0.000,0,20,-2.450,0,1
86,6154375,0.785714,This Jahablanoespanol is so stupid. The Donke...,0.114286,0.057143,0.585714,0.714286,0.014286,0.00,0.000000,...,this jahablanoespanol is so stupid the donkey...,jahablanoespanol stupid donkey's two weeks pre...,jahablanoespanol stupid donkey two week prepar...,jahablanoespanol stupid donkey's two week prep...,0.055,1,65,-2.923,1,1
93,6187983,0.600000,"Definitely a white person. Torn apart inside, ...",0.000000,0.000000,0.600000,0.300000,0.000000,0.00,0.000000,...,definitely a white person torn apart inside by...,definitely white person torn apart inside cons...,definit white person torn apart inside constan...,definit whit person torn apart inside const id...,0.029,0,51,-6.275,0,1
97,5223210,0.600000,Heterosexuals_tend_to_find_gay_sex_icky._It_is...,0.100000,0.200000,0.600000,0.400000,0.000000,0.00,0.000000,...,heterosexuals_tend_to_find_gay_sex_icky._it_is...,heterosexuals_tend_to_find_gay_sex_icky._it_is...,heterosexuals_tend_to_find_gay_sex_icky._it_is...,heterosexuals_tend_to_find_gay_sex_icky._it_is...,0.022,0,1,-134.000,0,1
128,5431394,0.700000,The irony is that Scalise campaigns against ga...,0.000000,0.000000,0.700000,0.100000,0.400000,0.00,0.000000,...,the irony is that scalise campaigns against ga...,irony scalise campaigns gays gay marriage save...,ironi scalis campaign gay gay marriage save le...,irony scal campaign gay gay marriage sav lesb ...,0.028,0,53,-2.264,0,1


Because we are unable to train an NB model on categorical (text) and continuous (numerical) data at the same time, our action plan changed to running two independent models for each type of data and then running a thrid NB model on the resulting predict_proba from the other two trained models.

In [34]:
def run_model(model_df, train_perc=.80, addtl_feats =[''], model_type = "Multi", 
              num_iter = 10, should_print=False, see_inside=False, comments="comment_text",
             target='cat'):
    
    train_start = 0
    train_end = round(model_df.shape[0]*train_perc) 

    test_start = train_end
    test_end = model_df.shape[0]
    
    X_all = model_df[comments].values
    y_all = model_df[target].values
    
    # tokenizing text
#     count_vect = CountVectorizer()
#     X_all_counts = count_vect.fit_transform(X_all.astype('U'))
    #print(X_all_counts.shape)

    # calculating frequencies
    tfidf_vectorizer = TfidfVectorizer(use_idf=True)
    fitted_vectorizer=tfidf_vectorizer.fit(model_df[comments])
    X_all_tfidf =  fitted_vectorizer.transform(model_df[comments])


    print(X_all_tfidf.shape)
    
    if addtl_feats != ['']: # combine non-text and text features if necessary
        print("here")
#         others_all = model_df[addtl_feats].values.reshape(-1,1)

        others_all = model_df[addtl_feats].values.reshape(-1,len(addtl_feats))
        #print(others_all)
        newfeatures_all = sparse.hstack((X_all_tfidf, others_all.astype(float))).tocsr()
    else:
        newfeatures_all = X_all_tfidf
    
    
    X_train = newfeatures_all[train_start:train_end]
    y_train = model_df[train_start:train_end][target].values
    y_train=y_train.astype('int')
    

    X_test = newfeatures_all[test_start:test_end]
    y_test = model_df[test_start:test_end][target].values
    
    
    
    
    if model_type == 'Multi':
        clf = MultinomialNB().fit(X_train, y_train)
    if model_type == "Gauss":
        clf = GaussianNB().fit(X_train, y_train) 
        
    preds_for_train = clf.predict(X_train)
    
    
   
    predicted = clf.predict(X_test)
    accuracy = np.mean(predicted == y_test)
    
    output = model_df[test_start:test_end]
    output['predicted'] = predicted
    output['y_test'] = y_test
    output['accuracy'] = output.predicted == output.y_test
    

#     y_scores_sorted, y_true_sorted = joint_sort_descending(np.array(y_scores), np.array(y_true))
#     precision = precision_score(y_true_sorted, preds)


    if should_print == True:

        print("The accuracy on the test set is {}%.".format(round(accuracy*100,2)))    
    
    if see_inside == True:
        return clf, accuracy, X_all_counts, X_all_tfidf
    else:
        return clf, accuracy, preds_for_train, predicted, output


In [35]:
clf1, accuracy, preds_for_train, predicted , output = run_model(sample_df, comments = "cleaned_lancaster", should_print=False)

print("The unique values predicted in the training set include :" + str(np.unique(preds_for_train)))
print("The unique values predicted in the test set include :" + str(np.unique(predicted)))

(8816, 17801)
The unique values predicted in the training set include :[0 1]
The unique values predicted in the test set include :[0 1]


In [36]:
output.y_test.value_counts()

0    893
1    870
Name: y_test, dtype: int64

In [37]:
output.predicted.value_counts()

1    1072
0     691
Name: predicted, dtype: int64

In [38]:
accuracy

0.8218944980147476

In [39]:
targets = output[output.y_test == 1]
targets[targets.accuracy == True].shape[0] / targets.shape[0]

0.9356321839080459

In [28]:
output[output.y_test == 0].accuracy.value_counts()

True     618
False    275
Name: accuracy, dtype: int64

In [44]:
targets[targets.identity_attack > .8].accuracy.value_counts()

True    7
Name: accuracy, dtype: int64

In [29]:
for text in ['cleaned_w_stopwords', 'cleaned_no_stem', 'cleaned_porter',
       'cleaned_lancaster']:
    
    print(text)
    
    clf, accuracy, preds_for_train, predicted, output = run_model(sample_df, comments = text, should_print=False)
    
    print("The unique values predicted for the training set include :" + str(np.unique(preds_for_train)))
    print("The unique values predicted for the test set include :" + str(np.unique(predicted)))
    
    targets = output[output.y_test == 1]
    target_accuracy = targets[targets.accuracy == True].shape[0] / targets.shape[0]
    
    print("Accuracy: {} , Target Accuracy: {}".format(accuracy, target_accuracy))
    print()



cleaned_w_stopwords
(8816, 20913)
The unique values predicted for the training set include :[0 1]
The unique values predicted for the test set include :[0 1]
Accuracy: 0.8162223482699943 , Target Accuracy: 0.9436781609195403

cleaned_no_stem
(8816, 20910)
The unique values predicted for the training set include :[0 1]
The unique values predicted for the test set include :[0 1]
Accuracy: 0.8241633579126489 , Target Accuracy: 0.9494252873563218

cleaned_porter
(8816, 18761)
The unique values predicted for the training set include :[0 1]
The unique values predicted for the test set include :[0 1]
Accuracy: 0.8269994327850255 , Target Accuracy: 0.9494252873563218

cleaned_lancaster
(8816, 17801)
The unique values predicted for the training set include :[0 1]
The unique values predicted for the test set include :[0 1]
Accuracy: 0.8218944980147476 , Target Accuracy: 0.9356321839080459

